## 7.4 GalleryApp

W tej aplikacji utworzymy prostą aplikację pełniącą rolę galerii zdjęć wykonanych aparatem. Będzie ona zawierać upoważnienia, bazę danych `SQLite`, dane będziemy wyświetlać w `RecyclerView`, obsługa aparatu odbędzie się za pomocą `Implicit Intent` - czyli otworzymy aplikację obsługującą aparat, wykonamy zdjęcie i prześlemy je do naszej aplikacji, następnie dodamy je do bazy danych i wyświetlimy w `RecyclerView`. Aplikacja będzie zawierać `Jetpack Navigation` oraz `Bottom Navigation`.

<table><tr><td><img src="https://media0.giphy.com/media/G5SmlOFRHlILBw9R0p/giphy.gif?cid=790b7611e3c8a0777e1e09d8e312633955f2b237e8abe275&rid=giphy.gif&ct=g" width="200" /></td><td><img src="https://media2.giphy.com/media/WxnetLg3yS6HddItSS/giphy.gif?cid=790b76112cb28038325c4acc25284dfa06b0788beca930d1&rid=giphy.gif&ct=g" width="200" /></td><td><img src="https://media0.giphy.com/media/oHuY0UmB6ZQY7Q7h31/giphy.gif?cid=790b7611d9bf36c59e0682886abc760a9fbec4b5db0f4f2e&rid=giphy.gif&ct=g" width="200" /></td></tr></table>

### **Bottom Navigation**

Rozpocznijmy od dodania dwóch fragmentów oraz nawigacji. Dodajmy dwa puste fragmenty `GalleryFragment` oraz `AddPictureFragment` oraz `navigation`

In [ ]:
<navigation xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    xmlns:tools="http://schemas.android.com/tools"
    android:id="@+id/navigation"
    app:startDestination="@id/galleryFragment">

    <fragment
        android:id="@+id/addPictureFragment"
        android:name="pl.udu.uwr.pum.galleryappjava.fragments.AddPictureFragment"
        android:label="fragment_add_picture"
        tools:layout="@layout/fragment_add_picture" />
    <fragment
        android:id="@+id/galleryFragment"
        android:name="pl.udu.uwr.pum.galleryappjava.fragments.GalleryFragment"
        android:label="fragment_gallery"
        tools:layout="@layout/fragment_gallery" />
</navigation>

`GalleryFragment` ustawiamy jako domowy. Następnie dodajmy `menu` dla `Bottom Navigation`

In [ ]:
<menu xmlns:android="http://schemas.android.com/apk/res/android">
    <item
        android:id="@id/galleryFragment"
        android:icon="@drawable/ic_gallery"
        android:title="Gallery" />
    <item
        android:id="@id/addPictureFragment"
        android:icon="@drawable/ic_add"
        android:title="Add" />
</menu>

Dodajmy `FragmentContainer` i `BottomNavigationView` do layoutu głównej aktywności

In [ ]:
<LinearLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:tools="http://schemas.android.com/tools"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    android:orientation="vertical"
    tools:context=".MainActivity">

    <androidx.fragment.app.FragmentContainerView
        android:id="@+id/nav_host_fragment"
        android:name="androidx.navigation.fragment.NavHostFragment"
        android:layout_width="match_parent"
        android:layout_height="0dp"
        android:layout_weight="1"
        app:defaultNavHost="true"
        app:navGraph="@navigation/navigation"/>

    <com.google.android.material.bottomnavigation.BottomNavigationView
        android:id="@+id/bottom_nav_view"
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        app:menu="@menu/bottom_menu" />

</LinearLayout>

Przejdźmy do `MainActivity` i połączmy nawigację

In [ ]:
public class MainActivity extends AppCompatActivity {

    @Override
    protected void onCreate(Bundle savedInstanceState) {
        super.onCreate(savedInstanceState);
        ActivityMainBinding binding = ActivityMainBinding.inflate(getLayoutInflater());
        setContentView(binding.getRoot());

        NavHostFragment navHostFragment = (NavHostFragment) getSupportFragmentManager()
                .findFragmentById(R.id.nav_host_fragment);
        if (navHostFragment != null) {
            NavController navController = NavHostFragment.findNavController(navHostFragment);
            NavigationUI.setupWithNavController(binding.bottomNavView, navController);
        }
    }
}

### **Permission**

Dodajmy upoważnienie na wykorzystanie aparatu do aplikacji. Przejdźmy do `AndroidManifest.xml` i dodajmy odpowiedni wpis

In [ ]:
<uses-permission android:name="android.permission.CAMERA" />

W layoucie `AddPictureFragment` dodamy dwa przyciski odpowiedzialne za wykonanie zdjęcia (sprawdzenie upoważnienia oraz otworzenie aplikacji obsługującej aparat) oraz za zapis do bazy danych. Mamy również `EditText` w który będziemy wpisywać tytuł zdjęcia oraz `ImageView` do którego będziemy przekazywać wykonane zdjęcie.

In [ ]:
<LinearLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    android:orientation="vertical">

    <EditText
        android:id="@+id/edit_text_title"
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:layout_margin="20dp"
        android:textSize="24sp"
        android:hint="Title"
        android:inputType="textCapWords"
        android:autofillHints="title" />

    <LinearLayout
        android:layout_width="wrap_content"
        android:layout_height="0dp"
        android:layout_weight="1"
        android:gravity="center"
        android:layout_gravity="center"
        android:orientation="vertical">

        <ImageView
            android:id="@+id/image_view_picture"
            android:layout_width="300dp"
            android:layout_height="300dp"
            android:layout_margin="40dp"
            android:contentDescription="picture" />

    </LinearLayout>

    <LinearLayout
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:orientation="horizontal">

    <Button
        android:id="@+id/button_camera"
        android:layout_width="0dp"
        android:layout_weight="1"
        android:layout_margin="25dp"
        android:layout_height="wrap_content"
        android:layout_gravity="center"
        android:text="otwórz aparat"
        android:gravity="center"/>

    <Button
        android:id="@+id/button_save_picture"
        android:layout_width="0dp"
        android:layout_weight="1"
        android:layout_height="wrap_content"
        android:layout_margin="25dp"
        android:gravity="center"
        android:layout_gravity="center"
        android:text="zapisz"/>
    </LinearLayout>
</LinearLayout>

Przy naciśnięciu przycisku **zapisz** wpierw sprawdzimy czy aplikacja posiada uprawnienie, następnie wyślemy odpowiedni `Intent`. W pierwszym kroku dodajmy `ViewBinding`

In [ ]:
private FragmentAddPictureBinding binding;

@Override
public View onCreateView(@NonNull LayoutInflater inflater, ViewGroup container,
                         Bundle savedInstanceState) {
    binding = FragmentAddPictureBinding.inflate(inflater, container, false);
    return binding.getRoot();
}

Zacznijmy od dwóch `ActivityResultLauncher`
- `requestCameraPermissionLauncher` - implementuje metodę `RequestPermission`, wykorzystamy do otworzenia dialogu z możliwością udzielenia upoważnienia
- `resultLauncherCamera` - implementacja metody `StartActivityForResult`, wykorzystamy przy wysłaniu intentu z żądaniem wykonania zdjęcia - dzięki tej metodzie możemy obsłużyć wartość zwrotną.

In [ ]:
ActivityResultLauncher<String> requestCameraPermissionLauncher = registerForActivityResult(
        new ActivityResultContracts.RequestPermission(), isGranted -> {
            if (isGranted) {
                launchCamera();
            }
        });

Jeżeli posiadamy odpowiednie upoważnienie wykonujemy metodę `launchCamera`, którą zaimplementujemy nieco później.

Teraz przejdźmy do `resultLauncherCamera`

In [ ]:
ActivityResultLauncher<Intent> resultLauncherCamera = registerForActivityResult(
        new ActivityResultContracts.StartActivityForResult(),
        result -> {});

Jeżeli zdjęcie zostanie wykonane prawidłowo i dostaniemy dane zwrotne (sprawdzamy w warunku), odbieramy dane za pomocą `Intent`

In [ ]:
if (result.getResultCode() == Activity.RESULT_OK) {
    Intent data = result.getData();

Następnie musimy je rozpakować, dane w tym przypadku otrzymujemy w postaci `Bitmap`

In [ ]:
if (result.getResultCode() == Activity.RESULT_OK) {
    Intent data = result.getData();
    Bitmap imageBitmap;
    if (data != null) {
        imageBitmap = (Bitmap) data.getExtras().get("data");
        binding.imageViewPicture.setImageBitmap(imageBitmap);
    }
}

Klucz "data", wykorzystany w metodzie `get` jest standardową nazwą nadaną automatycznie. Następnie ustawiamy otrzymaną bitmapę na `ImageView` za pomocą metody `setImageBitmap`. 

Dodajmy metodę `openCamera`, którą wywołamy jako `onClick` przycisku wykonującego zdjęcie. Mamy trzy możliwości
- upoważnienie zostało nadane - wywołujemy metodę `launchCamera`
- upoważnienie zostało odrzucone - pokazujemy `Rationale`
- aplikacja jest świeżo zainstalowana i włączona pierwszy raz - pokazujemy dialog z możliwością nadania upoważnienia

Te trzy opcje chcemy obsłużyć

In [ ]:
private void openCamera(){
    if (ContextCompat.checkSelfPermission(requireContext(), Manifest.permission.CAMERA)
            == PackageManager.PERMISSION_GRANTED )
        launchCamera(); // włączam aplikację przez implicit intent
    else if (ActivityCompat.shouldShowRequestPermissionRationale(requireActivity(), 
                                                                 Manifest.permission.CAMERA))
        showMessageOKCancel(getString(R.string.rationale_camera)); // Rationale
    else
        requestCameraPermissionLauncher.launch(Manifest.permission.CAMERA); // proszę o upoważnienie
}

Dodajmy implementację metody `showMessageOkCancel`

In [ ]:
private void showMessageOKCancel(String message) {
    new AlertDialog.Builder(requireContext())
            .setMessage(message)
            .setPositiveButton("OK", (dialog, which) -> {  // jeżeli ok proszę o upoważnienie
                requestCameraPermissionLauncher.launch(Manifest.permission.CAMERA);
                dialog.dismiss();
            })
            .setNegativeButton("Cancel", null) // jeżeli nie to nic nie robię
            .create()
            .show();
}

Pozostaje metoda `launchCamera` wysyłająca intent z prośbą o wykonanie zdjęcia

In [ ]:
private void launchCamera(){
    Intent intent = new Intent(MediaStore.ACTION_IMAGE_CAPTURE);
    resultLauncherCamera.launch(intent);
}

Dodajmy obsługę przycisku `buttonCamera` w metodzie `onViewCreated`

In [ ]:
binding.buttonCamera.setOnClickListener(v -> {
    openCamera();
});

### **Zapis do bazy**

W następnym kroku zapiszemy zdjęcie w bazie danych, rozpocznijmy od określenia modelu danych

In [ ]:
public class PictureModel {
    private int id = 0;
    private final String title;
    private final String image;

    public PictureModel(String title, String image) {
        this.title = title;
        this.image = image;
    }

    public PictureModel(int id, String title, String image){
        this(title, image);
        this.id = id;
    }

    public String getTitle() {
        return title;
    }

    public String getImage() {
        return image;
    }
}

Będziemy przechowywać nazwę zdjęcia i jego ścieżkę dostępu jako `String`. Przejdźmy do klasy `DBHandler`. Dodajmy podstawowe metody i pola

In [ ]:
public class DBHandler extends SQLiteOpenHelper {

    private static final int DATABASE_VERSION = 1;
    private static final String DATABASE_NAME = "galleryDBJava";
    private static final String TABLE_GALLERY = "GalleryTablString";

    private static final String KEY_ID = "_id";
    private static final String KEY_TITLE = "title";
    private static final String KEY_IMAGE = "image";

    public DBHandler(@Nullable Context context){
        super(context, DATABASE_NAME, null, DATABASE_VERSION);
    }

    @Override
    public void onCreate(SQLiteDatabase db) {
        String CREATE_GALLERY_TABLE = "CREATE TABLE " +
                TABLE_GALLERY +
                "(" +
                KEY_ID + " " +
                "INTEGER PRIMARY KEY," +
                KEY_TITLE +
                " TEXT," +
                KEY_IMAGE +
                " TEXT" +
                ")";

        db.execSQL(CREATE_GALLERY_TABLE);
    }

    @Override
    public void onUpgrade(SQLiteDatabase db, int oldVersion, int newVersion) {
        db.execSQL("DROP TABLE IF EXISTS " + TABLE_GALLERY);
        onCreate(db);
    }
}

Następnie dodajmy metodę dodającą wpis do bazy

In [ ]:
public long addToGallery(PictureModel singleItem){
    SQLiteDatabase db = this.getWritableDatabase();

    ContentValues contentValues = new ContentValues();
    contentValues.put(KEY_TITLE, singleItem.getTitle());
    contentValues.put(KEY_IMAGE, singleItem.getImage());

    long result = db.insert(TABLE_GALLERY, null, contentValues);
    db.close();
    return result;
}

Zmienna `result` posłuży do określenia poprawności wykonania operacji.

Drugą metodą będzie `getAllItems` zwracająca listę wszystkich elementów, lub pustą listę

In [ ]:
public ArrayList<PictureModel> getAllItems(){
    ArrayList<PictureModel> itemList = new ArrayList<>();

    String selectQuery = "SELECT * FROM " + TABLE_GALLERY;

    SQLiteDatabase db = this.getReadableDatabase();

    try{
        Cursor cursor = db.rawQuery(selectQuery, null);
        if(cursor.moveToFirst()){
            do{
                PictureModel place = new PictureModel(
                        cursor.getInt(0),
                        cursor.getString(1),
                        cursor.getString(2)
                );
                itemList.add(place);
            } while (cursor.moveToNext());
        }
        cursor.close();
    } catch (SQLiteException e){
        e.printStackTrace();
        return new ArrayList<>();
    }
    return itemList;
}

Wróćmy do klasy `AddPictureFragment` - chcemy zapisać ścieżkę dostępu do pliku, więc zdefiniujmy zmienną `Uri`

In [ ]:
private Uri pictureAbsolutePath;

W obsłudze `buttonSavePicture` wpierw obsłużmy błędy, w tym celu zdefiniujmy metodę `checkForErrors`

In [ ]:
private boolean checkForErrors(){
    if (binding.editTextTitle.getText().toString().isEmpty())
        return true;
    return pictureAbsolutePath == null;
}

Wyświetlimy `Toast` jeżeli pojawią się błędy

In [ ]:
binding.buttonSavePicture.setOnClickListener(v -> {
    if (checkForErrors())
        Toast.makeText(
        getContext(), 
        getString(R.string.error_imageView), 
        Toast.LENGTH_LONG
    ).show();

W przeciwnym razie tworzymy nowy wpis na podstawie zdefiniowanego modelu

In [ ]:
    else {
            PictureModel item = new PictureModel(
                binding.editTextTitle.getText().toString(),
                pictureAbsolutePath.toString()
        );

i dodajemy do bazy

In [ ]:
            DBHandler dbHandler = new DBHandler(requireContext());
            long addItemResult = dbHandler.addToGallery(item);

Na koniec wyświetlamy informację o powodzeniu operacji

In [ ]:
            if(addItemResult > 0)
                Toast.makeText(getContext(), "SUCCESS", Toast.LENGTH_SHORT).show();
        }
    });
}

Musimy jeszcze zapisać plik lokalnie na dysku (w bazie przechowujemy tylko ścieżkę dodstępu), zdefiniujmy metodę `saveImage`

In [ ]:
private Uri saveImage(Bitmap bitmap) {

Utwórzmy plik i posłużmy się metodą `getDir` klasy `Context` - jeżeli katalog o zadanej nazwie nie istnieje zostaje on utworzony, jeżeli istnieje nowy plik zostanie do niego dodany. Jest to katalog w którym aplikacja posiada uprawnienia do zapisu i odczytu własnych danych.

In [ ]:
File file = requireContext().getDir("myGalleryJava", Context.MODE_PRIVATE);

Następnie tworzymy plik - tutaj nazwą pliku będzie uniwersalny unikalny identyfikator `UUID` wygenerowany losowo

In [ ]:
file = new File(file, UUID.randomUUID().toString() + ".jpg");

Oraz zapisujemy otrzymaną bitmapę

In [ ]:
try {
    OutputStream stream = new FileOutputStream(file);
    bitmap.compress(Bitmap.CompressFormat.JPEG, 100, stream);
    stream.flush();
    stream.close();
} catch (IOException e) {
    e.printStackTrace();
}

Metoda zwraca ścieżkę absolutną jako `Uri`

In [ ]:
return Uri.parse(file.getAbsolutePath());

Metodę `saveImage` wykonujemy przy odebraniu danych - `resultLauncherCamera`

In [ ]:
ActivityResultLauncher<Intent> resultLauncherCamera = registerForActivityResult(
        new ActivityResultContracts.StartActivityForResult(),
        result -> {
            if (result.getResultCode() == Activity.RESULT_OK) {
                Intent data = result.getData();
                Bitmap imageBitmap;
                if (data != null) {
                    imageBitmap = (Bitmap) data.getExtras().get("data");
                    binding.imageViewPicture.setImageBitmap(imageBitmap);
                    pictureAbsolutePath = saveImage(imageBitmap); // zapis pliku oraz ścieżki
                }
            }
        });

### **RecyclerView**

W `GalleryFragment` będzie znajdował się `RecyclerView`, więc rozpocznijmy od adaptera

In [ ]:
public class GalleryAdapter extends RecyclerView.Adapter<GalleryAdapter.ViewHolder> {

    private final ArrayList<PictureModel> pictures;

    public GalleryAdapter(ArrayList<PictureModel> pictures){
        this.pictures = pictures;
    }
    @NonNull
    @Override
    public ViewHolder onCreateViewHolder(@NonNull ViewGroup parent, int viewType) {
        return new ViewHolder(ItemViewBinding.inflate(
                LayoutInflater.from(parent.getContext()), parent, false));
    }

    @Override
    public void onBindViewHolder(@NonNull ViewHolder holder, int position) {
        PictureModel item = pictures.get(position);
        holder.bind(item);
    }

    @Override
    public int getItemCount() {
        return pictures.size();
    }

    public static class ViewHolder extends RecyclerView.ViewHolder {
        private final ItemViewBinding itemBinding;
        public ViewHolder(@NonNull ItemViewBinding itemBinding) {
            super(itemBinding.getRoot());
            this.itemBinding = itemBinding;
        }

        public void bind(PictureModel item){
            itemBinding.textViewTitle.setText(item.getTitle());
            itemBinding.rcImageView.setImageURI(Uri.parse(item.getImage()));
        }
    }
}

dodajmy `RecyclerView` do layoutu fragmentu

In [ ]:
<FrameLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    tools:context=".fragments.GalleryFragment">

    <androidx.recyclerview.widget.RecyclerView
        android:id="@+id/recycler"
        android:layout_width="match_parent"
        android:layout_height="match_parent" />

</FrameLayout>

Ostatnim krokiem będzie dodanie `RecyclerView` do `GalleryFragment`

In [ ]:
public class GalleryFragment extends Fragment {

    private FragmentGalleryBinding binding;

    @Override
    public View onCreateView(@NonNull LayoutInflater inflater, ViewGroup container,
                             Bundle savedInstanceState) {
        binding = FragmentGalleryBinding.inflate(inflater, container, false);
        return binding.getRoot();
    }

    @Override
    public void onViewCreated(@NonNull View view, @Nullable Bundle savedInstanceState) {
        super.onViewCreated(view, savedInstanceState);
        DBHandler dbHandler = new DBHandler(requireContext());
        binding.recycler.setLayoutManager(new LinearLayoutManager(requireContext()));
        binding.recycler.setAdapter(new GalleryAdapter(dbHandler.getAllItems()));
    }
}

Możemy przetestować aplikację

<table><tr><td><img src="https://media0.giphy.com/media/G5SmlOFRHlILBw9R0p/giphy.gif?cid=790b7611e3c8a0777e1e09d8e312633955f2b237e8abe275&rid=giphy.gif&ct=g" width="150" /></td><td><img src="https://media2.giphy.com/media/WxnetLg3yS6HddItSS/giphy.gif?cid=790b76112cb28038325c4acc25284dfa06b0788beca930d1&rid=giphy.gif&ct=g" width="150" /></td><td><img src="https://media0.giphy.com/media/oHuY0UmB6ZQY7Q7h31/giphy.gif?cid=790b7611d9bf36c59e0682886abc760a9fbec4b5db0f4f2e&rid=giphy.gif&ct=g" width="150" /></td></tr></table>